In [ ]:
import pandas as pd
import numpy as np
import helper as h

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

In [ ]:
df_train = pd.read_hdf("../input/train_data.h5")
df_train['price'] = df_train['price'].map(h.parse_price)

df_test = pd.read_hdf("../input/test_data.h5")

print(df_train.shape, df_test.shape)

(22732, 8) (11448, 7)


In [ ]:
df = pd.concat([df_train, df_test])

In [ ]:
df.head()

,geo_block,breadcrumbs,price,owner,params,date,user_block,id
0,"[г. Москва, Лианозово, г. Москва, Лианозово]","[Москва, Лианозово, м. Алтуфьево, МЦК Лихоборы]",5.402084,[],"{'Охрана:': 'закрытая территория', 'Тип здания...","[22 апреля, 12, (+1 за сегодня), Обновлено 18 ...","[<div class=""block-user__name"">НДВ Недвижимост...",34995
1,"[г. Москва, Ховрино, г. Москва, Ховрино]","[Москва, Ховрино, м. Ховрино, МЦК Коптево]",6.827990,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[10 мая, 2, (+1 за сегодня), Обновлено 16 мая]","[<div class=""block-user__name"">MR-Group</div>,...",15030
2,"[г. Москва, ул Лобачевского, г. Москва, ул Лоб...","[Москва, МЦК Кутузовская, ул. Лобачевского]",8.680000,"[ЖК ""Крылья"". Доп. скидка 5%!]","{'Лифт:': 'да', 'Охрана:': 'закрытая территори...","[20 марта, 12, (+2 за сегодня), Обновлено 18 мая]",[],37811
3,"[г. Москва, Ховрино, ул Дыбенко, г. Москва, Хо...","[Москва, Ховрино, м. Ховрино, МЦК Коптево, ул....",8.624435,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[29 апреля, 3, (+1 за сегодня), Обновлено 13 мая]","[<div class=""block-user__name"">MR-Group</div>,...",31685
4,"[г. Москва, Даниловский, г. Москва, Даниловский]","[Москва, Даниловский, м. Автозаводская, МЦК Ав...",29.500000,[],"{'Общая площадь:': '141.1 м²', 'Дата публикаци...","[15 декабря 2018, 30, (+1 за сегодня), Обновле...","[<div class=""block-user__name"">Наталья Вдовиче...",56215


<b>Смотрим на данные и видим, что данные живые и настоящие. Начинаем обрабатывать с первого столбца - geo_block. Так как видно что geo_block имеет повторяющие элементы, нужно от них избавиться, + так как мы смотрим цены в Москве, мы можем удалить первый элемент списка - г.Москва.

In [ ]:
def parse_geo_block(l):
    l = l[:(len(l)//2)]
    if len(l)>1: 
        l = l[1:]
    return " ".join(l)

In [ ]:
df.geo_block = df.geo_block.apply(lambda x: parse_geo_block(x))

<b>Теперь рассмотрим вторую колонку - breadcrumbs. С этого столбца можно достать много информации, а именно - district, metro, MCK и street, для этого напишем пару функций для обработки информации.

In [ ]:
def parse_breadcrumbs(l):
    if len(l)==1:
        return l[0]
    else:
        return l[1]
df['district'] = df.breadcrumbs.apply(lambda x: parse_breadcrumbs(x))

def parse_breadcrumbs_metro(l):
    for i in l:
        if 'м.' in i:
            return i
    return np.nan
df['metro'] = df.breadcrumbs.apply(lambda x: parse_breadcrumbs_metro(x))

def MCK(l):
    for i in l:
        if "МЦК" in i:
            return i
    return np.nan
df['MCK'] = df.breadcrumbs.apply(lambda x: MCK(x))

def street(l):
    for i in l:
        if 'ул.' in i or 'пр-кт.' in i:
            return i
    return np.nan
df['street'] = df.breadcrumbs.apply(lambda x: street(x))

In [ ]:
df.head()

,geo_block,breadcrumbs,price,owner,params,date,user_block,id,district,metro,MCK,street
0,Лианозово,"[Москва, Лианозово, м. Алтуфьево, МЦК Лихоборы]",5.402084,[],"{'Охрана:': 'закрытая территория', 'Тип здания...","[22 апреля, 12, (+1 за сегодня), Обновлено 18 ...","[<div class=""block-user__name"">НДВ Недвижимост...",34995,Лианозово,м. Алтуфьево,МЦК Лихоборы,NaN
1,Ховрино,"[Москва, Ховрино, м. Ховрино, МЦК Коптево]",6.827990,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[10 мая, 2, (+1 за сегодня), Обновлено 16 мая]","[<div class=""block-user__name"">MR-Group</div>,...",15030,Ховрино,м. Ховрино,МЦК Коптево,NaN
2,ул Лобачевского,"[Москва, МЦК Кутузовская, ул. Лобачевского]",8.680000,"[ЖК ""Крылья"". Доп. скидка 5%!]","{'Лифт:': 'да', 'Охрана:': 'закрытая территори...","[20 марта, 12, (+2 за сегодня), Обновлено 18 мая]",[],37811,МЦК Кутузовская,NaN,МЦК Кутузовская,ул. Лобачевского
3,Ховрино ул Дыбенко,"[Москва, Ховрино, м. Ховрино, МЦК Коптево, ул....",8.624435,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[29 апреля, 3, (+1 за сегодня), Обновлено 13 мая]","[<div class=""block-user__name"">MR-Group</div>,...",31685,Ховрино,м. Ховрино,МЦК Коптево,ул. Дыбенко
4,Даниловский,"[Москва, Даниловский, м. Автозаводская, МЦК Ав...",29.500000,[],"{'Общая площадь:': '141.1 м²', 'Дата публикаци...","[15 декабря 2018, 30, (+1 за сегодня), Обновле...","[<div class=""block-user__name"">Наталья Вдовиче...",56215,Даниловский,м. Автозаводская,МЦК Автозаводская,NaN


**Переходим к самому вкусному, это столбец - params. В этой колонке находиться очень много полезной нам информации. Количественную информацию сразу же в int64 или float64. Некоторую информацию было решено перевести  категориальную.**

**Какие информация находится в params, я взял с starte_2**
<p>{'Лифт:': 2,
 'Возможна ипотека:': 2,
 'Мусоропровод:': 2,
 'Мебель на кухне:': 2,
 'Холодильник:': 2,
 'Интернет:': 2,
 'Мебель:': 2,
 'Телефон:': 2,
 'Свободная планировка:': 2,
 'Возможен торг:': 2,
 'Класс:': 2,
 'Телевизор:': 2,
 'Детская площадка:': 2,
 'Стиральная машина:': 2,
 'Тип объявления:': 3,
 'Вид из окна:': 3,
 'Тип дома:': 3,
 'Тип санузла:': 4,
 'Тип комнат:': 4,
 'Предоплата:': 4,
 'Тип объекта:': 5,
 'Класс жилья:': 5,
 'Покрытие пола:': 5,
 'Тип перекрытия:': 5,
 'Комиссия агенту:': 6,
 'Тип балкона:': 6,
 'Ремонт:': 7,
 'Тип фундамента:': 7,
 'Этап строительства:': 8,
 'Количество комнат:': 13,
 'Наибольшее количество этажей:': 13,
 'Количество подъездов:': 14,
 'Тип здания:': 15,
 'Наименьшее количество этажей:': 19,
 'Количество корпусов:': 35,
 'Серия:': 60,
 'Высота потолков:': 68,
 'Сдача:': 86,
 'Год постройки:': 91,
 'Количество жилых помещений:': 94,
 'Количество этажей:': 99,
 'Охрана:': 110,
 'Этажность:': 110,
 'Управляющая компания:': 112,
 'Застройщик:': 220,
 'Дата  обновления:': 227,
 'Количество квартир:': 352,
 'Парковка:': 388,
 'Площадь кухни:': 424,
 'Новостройка:': 610,
 'Дата публикации:': 698,
 'Жилая комната:': 896,
 'Этаж:': 1241,
 'Адрес:': 1296,
 'Общая площадь:': 1684}

In [ ]:
df['security'] = df.params.apply(lambda x: dict(x).get('Охрана:'))

In [ ]:
df['building_type'] = df.params.apply(lambda x: dict(x).get('Тип здания:'))

In [ ]:
df['object_type'] = df.params.apply(lambda x: dict(x).get('Тип объекта:'))

In [ ]:
df['number_of_buildings'] = df.params.apply(lambda x: np.nan if (dict(x).get('Количество корпусов:'))==None else int(dict(x).get('Количество корпусов:')))
df.number_of_buildings = df.number_of_buildings.fillna(df.number_of_buildings.mean()).astype('int64')

In [ ]:
df['ad_type'] = df.params.apply(lambda x: dict(x).get('Тип объявления:'))

In [ ]:
df['Developer'] = df.params.apply(lambda x: dict(x).get('Застройщик:'))

In [ ]:
df['total_area'] = df.params.apply(lambda x: float(dict(x).get('Общая площадь:').split(" ")[0])).astype('float64')

In [ ]:
df['month_of_publication'] = df.params.apply(lambda x: dict(x).get('Дата публикации:').split(' ')[1])

In [ ]:
df['rooms'] = df.params.apply(lambda x: np.nan if (dict(x).get('Количество комнат:'))==None else int(dict(x).get('Количество комнат:')))
df.rooms = df.rooms.fillna(df.rooms.mean().astype('int64'))

In [ ]:
df['view_from_the_window'] = df.params.apply(lambda x: dict(x).get('Вид из окна:'))

In [ ]:
df['parking'] = df.params.apply(lambda x: dict(x).get('Парковка:'))

In [ ]:
df['update_date'] = df.params.apply(lambda x: dict(x).get('Дата  обновления:'))

In [ ]:
df['Housing_class'] = df.params.apply(lambda x: dict(x).get('Класс жилья:'))

In [ ]:
df['floors'] = df.params.apply(lambda x: np.nan if (dict(x).get('Количество этажей:'))==None else int(dict(x).get('Количество этажей:')))
df.floors = df.floors.fillna(df.floors.mean().astype('int64'))

In [ ]:
df['agent_commission'] = df.params.apply(lambda x: dict(x).get('Комиссия агенту:'))

In [ ]:
df['height'] = df.params.apply(lambda x: np.nan if (dict(x).get('Высота потолков:'))==None else float(dict(x).get('Высота потолков:').split(' ')[0]))
df.height = df.height.fillna(df.height.mean().astype('float64'))

In [ ]:
df['floor'] = df.params.apply(lambda x: np.nan if (dict(x).get('Этаж:'))==None else int(dict(x).get('Этаж:').split('/')[0]))
df.floor = df.floor.fillna(df.floor.mean().astype('int64'))

In [ ]:
df['New_building'] = df.params.apply(lambda x: dict(x).get('Новостройка:'))

In [ ]:
df['number_of_apartments'] = df.params.apply(lambda x: -1 if (dict(x).get('Количество квартир:'))==None else dict(x).get('Количество квартир:')).astype('int64')


In [ ]:
def f(x):
    if x == 'да':
        return 1
    return 0
df['Mortgage'] = df.params.apply(lambda x: f(dict(x).get('Возможна ипотека:')))

In [ ]:
df['elevator']= df.params.apply(lambda x: 0 if dict(x).get('Лифт:')==None else 1).astype('int64')

In [ ]:
df['balcony'] = df.params.apply(lambda x: dict(x).get('Тип балкона:'))

In [ ]:
df['Furniture'] = df.params.apply(lambda x: 0 if dict(x).get('Мебель:')==None else 1).astype('int64')

In [ ]:
df['free_layout'] = df.params.apply(lambda x: 0 if dict(x).get('Свободная планировка:')==None else 1).astype('int64')

In [ ]:
df['type_home'] = df.params.apply(lambda x: dict(x).get('Тип дома:'))

In [ ]:
df['Playground'] = df.params.apply(lambda x: 0 if dict(x).get('Детская площадка:')==None else 1).astype('int64')

In [ ]:
df['kitchen_area'] = df.params.apply(lambda x: np.nan if dict(x).get('Площадь кухни:')==None else float(dict(x).get('Площадь кухни:').split(' ')[0]))
df.kitchen_area = df.kitchen_area.fillna(df.kitchen_area.mean().astype('float64'))

In [ ]:
def security(l):
    if l==None or l.upper=='НЕТ':
        return 0
    elif 'круглосуточное' in l.split(' ') or 'Круглосуточная' in l.split(' ') or 'круглосуточная' in l.split(' '):
        return 4
    elif 'консьерж' in l.split(' '):
        return 3
    elif 'видеонаблюдение' in l.split(' '):
        return 2
    elif 'закрытая' in l.split(' ') or 'огороженная' in l.split(' '):
        return 1
    else:
        return 0
df.security = df.security.apply(lambda x: security(x))

In [ ]:
def parking(l):
    if l==None:
        return 0
    elif 'подземная' in l.split(' ') or 'подземный' in l.split(' ') or 'крытая' in l.split(' '):
        return 1
    elif 'наземная' in l.split(' '):
        return 2
    else: 
        return 3
df.parking = df.parking.apply(lambda x: parking(x))

In [ ]:
def bulding(l):
    if l=='Монолитно-кирпичный' or l=='Монолитно-кирпичное' or l=='Монолитно-панельный' or l=='Монолитно-панельное':
        return 'Монолит'
    return l
df.building_type = df.building_type.apply(lambda x: bulding(x))

**Нужно удалить столбцы которые, нам не нужны. Так как нужную информацию мы с них уже достали или эти столбцы имеют мало уникальной или нужно информацию **

In [ ]:
id = df_test.id
df = df.drop(['user_block', 'id','breadcrumbs', 'owner', 'params', 'date'], axis=1)
df.head()

,geo_block,price,district,metro,MCK,street,security,building_type,object_type,number_of_buildings,...,New_building,number_of_apartments,Mortgage,elevator,balcony,Furniture,free_layout,type_home,Playground,kitchen_area
0,Лианозово,5.402084,Лианозово,м. Алтуфьево,МЦК Лихоборы,NaN,1,Монолитное,квартиры,1,...,"Апарт-комплекс Nord, м. Алтуфьево",163,0,0,None,0,0,None,0,15.020488
1,Ховрино,6.827990,Ховрино,м. Ховрино,МЦК Коптево,NaN,0,Монолитное,квартира,5,...,"ЖК Discovery (ЖК «Дискавери»), м. Ховрино",1251,1,1,None,0,0,None,0,15.020488
2,ул Лобачевского,8.680000,МЦК Кутузовская,NaN,МЦК Кутузовская,ул. Лобачевского,1,Монолитное,квартиры,3,...,"ЖК «Крылья», м. Раменки",1473,0,1,None,0,0,None,0,15.020488
3,Ховрино ул Дыбенко,8.624435,Ховрино,м. Ховрино,МЦК Коптево,ул. Дыбенко,0,Монолитное,квартира,5,...,"ЖК Discovery (ЖК «Дискавери»), м. Ховрино",1251,1,1,None,0,0,None,0,15.020488
4,Даниловский,29.500000,Даниловский,м. Автозаводская,МЦК Автозаводская,NaN,0,Монолитное,квартиры,9,...,None,-1,0,0,None,0,0,None,0,15.020488


In [ ]:
df.update_date = df.update_date.apply(lambda x: x.split(' ')[1] if x!=None else 'Nan')
df.rename(columns={'update_date': 'update_month'})
d = {}
for i in list(df.columns):
    d[i] = df[i].nunique()
d

{'geo_block': 977,
 'price': 12529,
 'district': 258,
 'metro': 196,
 'MCK': 31,
 'street': 419,
 'security': 5,
 'building_type': 11,
 'object_type': 5,
 'number_of_buildings': 34,
 'ad_type': 3,
 'Developer': 219,
 'total_area': 1684,
 'month_of_publication': 13,
 'rooms': 12,
 'view_from_the_window': 2,
 'parking': 4,
 'update_date': 4,
 'Housing_class': 4,
 'floors': 98,
 'agent_commission': 5,
 'height': 68,
 'floor': 89,
 'New_building': 609,
 'number_of_apartments': 352,
 'Mortgage': 2,
 'elevator': 2,
 'balcony': 5,
 'Furniture': 2,
 'free_layout': 2,
 'type_home': 2,
 'Playground': 2,
 'kitchen_area': 424}

Перед LabelEncoder проверим types наших столбцов 

In [ ]:
df.dtypes

geo_block                object
price                   float64
district                 object
metro                    object
MCK                      object
street                   object
security                  int64
building_type            object
object_type              object
number_of_buildings       int64
ad_type                  object
Developer                object
total_area              float64
month_of_publication     object
rooms                   float64
view_from_the_window     object
parking                   int64
update_date              object
Housing_class            object
floors                  float64
agent_commission         object
height                  float64
floor                   float64
New_building             object
number_of_apartments      int64
Mortgage                  int64
elevator                  int64
balcony                  object
Furniture                 int64
free_layout               int64
type_home                object
Playgrou

Делаем LabelEncoder. Вот ссылка на статью о LabelEncoder и OneHotEncoder.
https://habr.com/ru/post/456294/

In [ ]:
df.columns

Index(['geo_block', 'price', 'district', 'metro', 'MCK', 'street', 'security',
       'building_type', 'object_type', 'number_of_buildings', 'ad_type',
       'Developer', 'total_area', 'month_of_publication', 'rooms',
       'view_from_the_window', 'parking', 'update_date', 'Housing_class',
       'floors', 'agent_commission', 'height', 'floor', 'New_building',
       'number_of_apartments', 'Mortgage', 'elevator', 'balcony', 'Furniture',
       'free_layout', 'type_home', 'Playground', 'kitchen_area'],
      dtype='object')

In [ ]:
category = ['geo_block', 'district', 'metro', 'MCK', 'street',
       'security', 'building_type', 'object_type', 'rooms',
       'ad_type', 'Developer', 'month_of_publication',
       'view_from_the_window', 'parking', 'update_date', 'Housing_class',
        'agent_commission', 'New_building',
       'Mortgage', 'balcony', 'type_home']

In [ ]:
le = LabelEncoder()
for i in category:
    df[i] = le.fit_transform(df[i]).astype('int64')

In [ ]:
df.dtypes

geo_block                 int64
price                   float64
district                  int64
metro                     int64
MCK                       int64
street                    int64
security                  int64
building_type             int64
object_type               int64
number_of_buildings       int64
ad_type                   int64
Developer                 int64
total_area              float64
month_of_publication      int64
rooms                     int64
view_from_the_window      int64
parking                   int64
update_date               int64
Housing_class             int64
floors                  float64
agent_commission          int64
height                  float64
floor                   float64
New_building              int64
number_of_apartments      int64
Mortgage                  int64
elevator                  int64
balcony                   int64
Furniture                 int64
free_layout               int64
type_home                 int64
Playgrou

**Я решил, что если np.log некоторых фичей, это может улучшить точность модели. НО это я решил когда уже, запускал модели и понял, что лучше всего работает RandomForest, и решил посмотреть feature importance, именно поэтому я выбрал именно эти фичи.
P.S. feature importance будет представлен немного ниже.**

In [ ]:
def log_c(name):
    df[name] = np.log(df[name])
    df[name] = np.where(df[name] == -np.inf, 1, df[name])
for i in ['geo_block', 'district', 'street', 'New_building', 'number_of_apartments', 'rooms', 'MCK','floors', 'floor', 'height', 'total_area', 'kitchen_area']:
    log_c(i) 
    df[i] = df[i].fillna(df[i].mean())
#'geo_block', 'district', 'street', 'New_building', 'number_of_apartments', 'rooms', 'MCK','floors', 'floor', 'height', 'total_area', 'kitchen_area' == 2.35 RF

/opt/conda/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufu

Разделим на train и test. + ещё разделим train на X_train, X_test, y_train, y_test с помощью train_test_split

In [ ]:
df_train = df[ ~df["price"].isnull() ].copy()
df_test = df[ df["price"].isnull() ].copy()

df_train.shape, df_test.shape

((22732, 33), (11448, 33))

In [ ]:
df_test = df_test.drop('price', axis=1)

In [ ]:
X = df_train.drop('price', axis=1)
y = df_train.price

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=17)
y_train_log = np.log(y_train)

In [ ]:
df.head()

,geo_block,price,district,metro,MCK,street,security,building_type,object_type,number_of_buildings,...,New_building,number_of_apartments,Mortgage,elevator,balcony,Furniture,free_layout,type_home,Playground,kitchen_area
0,5.476464,5.402084,3.761200,5,2.772589,6.037871,1,5,2,1,...,0.693147,5.093750,0,0,5,0,0,2,0,2.709415
1,6.335054,6.827990,4.934474,177,2.564949,6.037871,0,5,1,5,...,3.610918,7.131699,1,1,5,0,0,2,0,2.709415
2,6.803505,8.680000,4.127134,196,2.708050,5.303305,1,5,2,3,...,5.572154,7.295056,0,1,5,0,0,2,0,2.709415
3,6.340359,8.624435,4.934474,177,2.564949,4.969813,0,5,1,5,...,3.610918,7.131699,1,1,5,0,0,2,0,2.709415
4,4.919981,29.500000,3.091042,1,1.000000,6.037871,0,5,2,9,...,6.411818,6.947143,0,0,5,0,0,2,0,2.709415


**Пришло время к обучению. Было использовано: Linear Regression, Dummy Regressor, Random Forest Regressor, KNN Regressor, Gradient Boosting, XGB, CatBoost. Ещё использовались PCA - Метод Главных Компонент и StandardScaler, но к сильному улучшению линейных моделей это не привело, а остальные модели показывали хуже score. В итоге лучшая модель это Random Forest, но так же XGB и Catboost показывали похожий score что и RF**

In [ ]:
logit = LinearRegression()
logit.fit(X_train, y_train)
mean_absolute_error(y_test, logit.predict(X_test))

12.559523689704186

In [ ]:
model = DummyRegressor(strategy="median")
model.fit(X, y)
y_pred = model.predict(X)

mean_absolute_error(y, y_pred)

10.446140640198838

In [ ]:
sc = StandardScaler()

In [ ]:
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [ ]:
logit = LinearRegression()
logit.fit(X_train_sc, y_train)

LinearRegression()

In [ ]:
mean_absolute_error(y_test, logit.predict(X_test_sc))

12.559523689704186

In [ ]:
model = DummyRegressor(strategy="median")
model.fit(X_train_sc, y_train)
y_pred = model.predict(X_test_sc)

mean_absolute_error(y_test, y_pred)

10.121870564782725

In [ ]:
rf1 = RandomForestRegressor()
rf1.fit(X_train, y_train_log)
mean_absolute_error(y_test, np.exp(rf1.predict(X_test)))

2.4117405793727498

In [ ]:
rf = RandomForestRegressor()
rf.fit(X, np.log(y))
#final submit 

RandomForestRegressor()

In [ ]:
pd.DataFrame({'name': X_train.columns, 'feature_importances_': rf1.feature_importances_}).sort_values('feature_importances_', ascending=0)

,name,feature_importances_
11,total_area,0.734667
17,Housing_class,0.049417
20,height,0.044971
23,number_of_apartments,0.022111
3,MCK,0.020378
10,Developer,0.019873
0,geo_block,0.017899
8,number_of_buildings,0.012611
18,floors,0.010342
2,metro,0.010040


In [ ]:
param = {'n_estimators': range(90, 120, 10), 'max_depth': range(2, 10,3)}
rf_cv = GridSearchCV(rf, param, n_jobs=-1, scoring='neg_mean_absolute_error', cv=5)
rf_cv.fit(X_train, y_train)
rf_cv.best_estimator_
mean_absolute_error(y_test, rf_cv.predict(X_test))

In [ ]:
knn = KNeighborsRegressor()
knn.fit(X_train_sc, y_train)
mean_absolute_error(y_test, knn.predict(X_test_sc))

4.140561015435884

In [ ]:
pca = PCA(0.90)
X_train_pca_sc = pca.fit_transform(X_train_sc)
X_test_pca_sc = pca.transform(X_test_sc)
X_train_pca_sc.shape, X_train_sc.shape

((15230, 23), (15230, 32))

In [ ]:
knn = KNeighborsRegressor()
knn.fit(X_train_pca_sc, y_train)
mean_absolute_error(y_test, knn.predict(X_test_pca_sc))

4.137091468221807

In [ ]:
logit.fit(X_train_pca_sc, y_train)
mean_absolute_error(y_test, logit.predict(X_test_pca_sc))

12.766303562273182

In [ ]:
rf_pca = RandomForestRegressor(random_state=17)
rf_pca.fit(X_train_pca_sc, y_train)
mean_absolute_error(y_test, rf_pca.predict(X_test_pca_sc))

4.051223091528077

In [ ]:
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train_log)
mean_absolute_error(y_test, np.exp(gbr.predict(X_test)))

3.3878870864050556

In [ ]:
model = xgb.XGBRegressor()
model.fit(X_train, y_train_log)
mean_absolute_error(y_test, np.exp(model.predict(X_test)))

2.4228318632207357

In [ ]:
cat = cb.CatBoostRegressor(loss_function='RMSE')
cat.fit(X_train, y_train_log)
mean_absolute_error(y_test, np.exp(cat.predict(X_test)))

Learning rate set to 0.063375
0:	learn: 0.6951902	total: 59.6ms	remaining: 59.5s
1:	learn: 0.6648651	total: 69.2ms	remaining: 34.5s
2:	learn: 0.6365353	total: 82.4ms	remaining: 27.4s
3:	learn: 0.6099497	total: 90.7ms	remaining: 22.6s
4:	learn: 0.5846125	total: 101ms	remaining: 20s
5:	learn: 0.5624760	total: 110ms	remaining: 18.2s
6:	learn: 0.5413386	total: 132ms	remaining: 18.8s
7:	learn: 0.5210771	total: 143ms	remaining: 17.8s
8:	learn: 0.5015718	total: 151ms	remaining: 16.6s
9:	learn: 0.4836221	total: 158ms	remaining: 15.7s
10:	learn: 0.4673336	total: 168ms	remaining: 15.1s
11:	learn: 0.4518118	total: 175ms	remaining: 14.4s
12:	learn: 0.4377345	total: 186ms	remaining: 14.1s
13:	learn: 0.4244947	total: 196ms	remaining: 13.8s
14:	learn: 0.4123639	total: 227ms	remaining: 14.9s
15:	learn: 0.4010468	total: 239ms	remaining: 14.7s
16:	learn: 0.3903916	total: 248ms	remaining: 14.4s
17:	learn: 0.3805140	total: 257ms	remaining: 14s
18:	learn: 0.3719113	total: 269ms	remaining: 13.9s
19:	learn: 

173:	learn: 0.1848963	total: 1.83s	remaining: 8.69s
174:	learn: 0.1846704	total: 1.84s	remaining: 8.67s
175:	learn: 0.1842922	total: 1.85s	remaining: 8.66s
176:	learn: 0.1840175	total: 1.86s	remaining: 8.65s
177:	learn: 0.1838071	total: 1.87s	remaining: 8.62s
178:	learn: 0.1834951	total: 1.88s	remaining: 8.61s
179:	learn: 0.1832664	total: 1.9s	remaining: 8.66s
180:	learn: 0.1829744	total: 1.91s	remaining: 8.64s
181:	learn: 0.1825872	total: 1.92s	remaining: 8.63s
182:	learn: 0.1823855	total: 1.93s	remaining: 8.6s
183:	learn: 0.1821883	total: 1.93s	remaining: 8.58s
184:	learn: 0.1817103	total: 1.94s	remaining: 8.55s
185:	learn: 0.1813606	total: 1.95s	remaining: 8.54s
186:	learn: 0.1811438	total: 1.96s	remaining: 8.52s
187:	learn: 0.1810556	total: 1.97s	remaining: 8.49s
188:	learn: 0.1808972	total: 1.97s	remaining: 8.47s
189:	learn: 0.1806042	total: 2s	remaining: 8.54s
190:	learn: 0.1803565	total: 2.01s	remaining: 8.52s
191:	learn: 0.1803074	total: 2.02s	remaining: 8.49s
192:	learn: 0.180

344:	learn: 0.1514587	total: 3.47s	remaining: 6.6s
345:	learn: 0.1511067	total: 3.48s	remaining: 6.58s
346:	learn: 0.1510386	total: 3.49s	remaining: 6.57s
347:	learn: 0.1508909	total: 3.5s	remaining: 6.56s
348:	learn: 0.1507831	total: 3.51s	remaining: 6.55s
349:	learn: 0.1507002	total: 3.52s	remaining: 6.53s
350:	learn: 0.1505702	total: 3.52s	remaining: 6.51s
351:	learn: 0.1503861	total: 3.53s	remaining: 6.5s
352:	learn: 0.1502656	total: 3.54s	remaining: 6.49s
353:	learn: 0.1501495	total: 3.55s	remaining: 6.47s
354:	learn: 0.1500763	total: 3.56s	remaining: 6.46s
355:	learn: 0.1499510	total: 3.58s	remaining: 6.47s
356:	learn: 0.1498664	total: 3.58s	remaining: 6.45s
357:	learn: 0.1497113	total: 3.59s	remaining: 6.44s
358:	learn: 0.1494849	total: 3.6s	remaining: 6.42s
359:	learn: 0.1493506	total: 3.61s	remaining: 6.42s
360:	learn: 0.1492434	total: 3.62s	remaining: 6.4s
361:	learn: 0.1490569	total: 3.63s	remaining: 6.39s
362:	learn: 0.1489685	total: 3.63s	remaining: 6.38s
363:	learn: 0.148

508:	learn: 0.1338369	total: 5.1s	remaining: 4.92s
509:	learn: 0.1337210	total: 5.11s	remaining: 4.91s
510:	learn: 0.1336002	total: 5.12s	remaining: 4.9s
511:	learn: 0.1334863	total: 5.14s	remaining: 4.9s
512:	learn: 0.1333570	total: 5.16s	remaining: 4.89s
513:	learn: 0.1332515	total: 5.16s	remaining: 4.88s
514:	learn: 0.1331895	total: 5.17s	remaining: 4.87s
515:	learn: 0.1330917	total: 5.18s	remaining: 4.86s
516:	learn: 0.1330131	total: 5.19s	remaining: 4.84s
517:	learn: 0.1329432	total: 5.2s	remaining: 4.83s
518:	learn: 0.1328488	total: 5.2s	remaining: 4.82s
519:	learn: 0.1328148	total: 5.21s	remaining: 4.81s
520:	learn: 0.1327477	total: 5.22s	remaining: 4.8s
521:	learn: 0.1326633	total: 5.24s	remaining: 4.8s
522:	learn: 0.1326000	total: 5.25s	remaining: 4.79s
523:	learn: 0.1324715	total: 5.26s	remaining: 4.78s
524:	learn: 0.1324222	total: 5.27s	remaining: 4.76s
525:	learn: 0.1323121	total: 5.27s	remaining: 4.75s
526:	learn: 0.1322235	total: 5.28s	remaining: 4.74s
527:	learn: 0.13214

673:	learn: 0.1222483	total: 6.75s	remaining: 3.26s
674:	learn: 0.1221947	total: 6.75s	remaining: 3.25s
675:	learn: 0.1221164	total: 6.76s	remaining: 3.24s
676:	learn: 0.1220620	total: 6.77s	remaining: 3.23s
677:	learn: 0.1220098	total: 6.78s	remaining: 3.22s
678:	learn: 0.1219407	total: 6.79s	remaining: 3.21s
679:	learn: 0.1218734	total: 6.8s	remaining: 3.2s
680:	learn: 0.1217830	total: 6.81s	remaining: 3.19s
681:	learn: 0.1217320	total: 6.82s	remaining: 3.18s
682:	learn: 0.1216179	total: 6.83s	remaining: 3.17s
683:	learn: 0.1215458	total: 6.84s	remaining: 3.16s
684:	learn: 0.1215011	total: 6.84s	remaining: 3.15s
685:	learn: 0.1214203	total: 6.85s	remaining: 3.14s
686:	learn: 0.1213704	total: 6.86s	remaining: 3.13s
687:	learn: 0.1212964	total: 6.87s	remaining: 3.12s
688:	learn: 0.1212065	total: 6.88s	remaining: 3.1s
689:	learn: 0.1211593	total: 6.89s	remaining: 3.09s
690:	learn: 0.1211081	total: 6.91s	remaining: 3.09s
691:	learn: 0.1210534	total: 6.92s	remaining: 3.08s
692:	learn: 0.1

841:	learn: 0.1133225	total: 8.36s	remaining: 1.57s
842:	learn: 0.1132773	total: 8.38s	remaining: 1.56s
843:	learn: 0.1132318	total: 8.39s	remaining: 1.55s
844:	learn: 0.1131266	total: 8.4s	remaining: 1.54s
845:	learn: 0.1130740	total: 8.41s	remaining: 1.53s
846:	learn: 0.1130313	total: 8.42s	remaining: 1.52s
847:	learn: 0.1129740	total: 8.43s	remaining: 1.51s
848:	learn: 0.1129168	total: 8.44s	remaining: 1.5s
849:	learn: 0.1129085	total: 8.44s	remaining: 1.49s
850:	learn: 0.1128778	total: 8.45s	remaining: 1.48s
851:	learn: 0.1128365	total: 8.46s	remaining: 1.47s
852:	learn: 0.1128058	total: 8.48s	remaining: 1.46s
853:	learn: 0.1127317	total: 8.49s	remaining: 1.45s
854:	learn: 0.1127016	total: 8.5s	remaining: 1.44s
855:	learn: 0.1126647	total: 8.51s	remaining: 1.43s
856:	learn: 0.1126322	total: 8.52s	remaining: 1.42s
857:	learn: 0.1125954	total: 8.53s	remaining: 1.41s
858:	learn: 0.1125293	total: 8.53s	remaining: 1.4s
859:	learn: 0.1124814	total: 8.54s	remaining: 1.39s
860:	learn: 0.11

2.397999161885969